# Hypoxia Forecasting: Sliding-Window Classification (t+1, t+7, t+10) + LSTM

This notebook builds supervised sliding-window classifiers to forecast hypoxia at multiple lead times and includes an LSTM sequence model. Horizons:
- 1-day (t+1): uses t-1, t-2 (and differences)
- 7-day (t+7): uses t-1..t-7 (or weekly aggregates)
- 10-day (t+10): uses t-1..t-10 (or 2-week window)

Baselines: persistence and monthly climatology. Models: RandomForest (default) with optional LightGBM/XGBoost if available. We use temporal cross-validation to avoid leakage.

In [ ]:
# Imports & Configuration
import os, warnings, math, json, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import timedelta
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, roc_curve, precision_recall_curve, brier_score_loss
)
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
import joblib

warnings.filterwarnings('ignore')
np.random.seed(42)

# Paths (edit these as needed)
DATA_CSV = r"C:\\Users\\hafez\\MSU\\Research\\msGOM\\mssound\\bloom\\data\\processed\\hypoxia_timeseries.csv"
MODELS_DIR = r"C:\\Users\\hafez\\MSU\\Research\\msGOM\\mssound\\bloom\\data\\models_forecast"
os.makedirs(MODELS_DIR, exist_ok=True)

FEATURE_COLS = [
    'chlor_a','nflh','poc','sst','Rrs_412','Rrs_443','Rrs_469','Rrs_488',
    'Rrs_531','Rrs_547','Rrs_555','Rrs_645','Rrs_667','Rrs_678'
]
GROUP_COLS = ['lat','lon']  # per-pixel sequences
TARGET_COL = 'label'  # 0/1 hypoxia indicator
DATE_COL = 'date'

# Helper: metrics dict

def compute_metrics(y_true, y_prob, threshold=0.5):
    y_pred = (y_prob >= threshold).astype(int)
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'balanced_accuracy': balanced_accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, zero_division=0),
        'recall': recall_score(y_true, y_pred, zero_division=0),
        'f1': f1_score(y_true, y_pred, zero_division=0),
        'roc_auc': roc_auc_score(y_true, y_prob) if len(np.unique(y_true))>1 else np.nan,
        'brier': brier_score_loss(y_true, y_prob)
    }

In [ ]:
# 1) Load and Inspect Time Series Data
try:
    df = pd.read_csv(DATA_CSV)
    if DATE_COL in df.columns:
        df[DATE_COL] = pd.to_datetime(df[DATE_COL])
    df = df.sort_values([GROUP_COLS[0], GROUP_COLS[1], DATE_COL])
    print('Loaded:', df.shape, 'time span:', df[DATE_COL].min(), '→', df[DATE_COL].max())
    display(df.head())
    print('\nMissing values per column:')
    print(df.isna().sum())
except Exception as e:
    print('Please set DATA_CSV to a valid CSV path:', e)

In [ ]:
# 2) Preprocess and Engineer Features (differences, aggregates)
# - Basic NA handling
# - Optional scaling (per-feature global scaler)
# - First differences and rolling stats per pixel

# Simple NA fill: forward/backward per pixel, then global fill
df[FEATURE_COLS] = df.groupby(GROUP_COLS)[FEATURE_COLS].apply(lambda g: g.ffill().bfill()).reset_index(level=GROUP_COLS, drop=True)
df[FEATURE_COLS] = df[FEATURE_COLS].fillna(df[FEATURE_COLS].median())

# Fit scaler on features only (global)
scaler = MinMaxScaler()
df[[f'{c}_scaled' for c in FEATURE_COLS]] = scaler.fit_transform(df[FEATURE_COLS])

# Add first differences per pixel
for c in FEATURE_COLS:
    df[f'{c}_diff1'] = df.groupby(GROUP_COLS)[c].diff(1)
    df[f'{c}_diff1'] = df[f'{c}_diff1'].fillna(0.0)

# Optional rolling means (7-day) per pixel
for c in FEATURE_COLS:
    df[f'{c}_roll7'] = df.groupby(GROUP_COLS)[c].transform(lambda s: s.rolling(7, min_periods=1).mean())

In [ ]:
# 3) Create Sliding Windows for Multiple Horizons
# Build (X, y) with window size k and horizon h per pixel, then stack across pixels.

WINDOW_FEATURES = [
    *FEATURE_COLS,
    *[f'{c}_scaled' for c in FEATURE_COLS],
    *[f'{c}_diff1' for c in FEATURE_COLS],
]

from typing import Tuple

def build_supervised(df_in: pd.DataFrame, k: int, h: int) -> Tuple[pd.DataFrame, pd.Series, pd.Series]:
    """Return X, y, and timestamps for samples using sliding window size k and forecast horizon h (in steps).
    Assumes one row per (lat, lon, date)."""
    frames = []
    targets = []
    times = []
    for (lat, lon), g in df_in.groupby(GROUP_COLS):
        g = g.sort_values(DATE_COL).reset_index(drop=True)
        values = g[WINDOW_FEATURES].values
        y_arr = g[TARGET_COL].values
        for t in range(k, len(g) - h):
            X_win = values[t-k:t].ravel(order='C')
            y_t = y_arr[t + h]
            frames.append(X_win)
            targets.append(y_t)
            times.append(g.loc[t + h, DATE_COL])
    X = pd.DataFrame(frames)
    y = pd.Series(targets)
    ts = pd.Series(times)
    return X, y, ts

# Convenience: pick k based on horizon
H_K = {1: 2, 7: 7, 10: 10}

In [ ]:
# 4) Train/Validation/Test Split by Time
# We'll use the last 20% time for test, prior 20% for validation, rest for train.

def temporal_split(ts: pd.Series, test_frac=0.2, val_frac=0.2):
    ts_sorted = ts.sort_values().reset_index(drop=True)
    n = len(ts_sorted)
    t_test = ts_sorted.iloc[int((1 - test_frac) * n)]
    t_val = ts_sorted.iloc[int((1 - test_frac - val_frac) * n)]
    return t_val, t_test


def split_by_time(X, y, ts, t_val, t_test):
    train_idx = ts < t_val
    val_idx = (ts >= t_val) & (ts < t_test)
    test_idx = ts >= t_test
    return (X[train_idx], y[train_idx]), (X[val_idx], y[val_idx]), (X[test_idx], y[test_idx])

In [ ]:
# 5–7) Train Classifiers per Horizon (1, 7, 10 days)
from scipy.stats import randint, uniform

RFC_PARAM_DIST = {
    'n_estimators': randint(100, 400),
    'max_depth': randint(3, 20),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 8),
    'max_features': ['sqrt', 'log2', None]
}

results = []

for h in [1, 7, 10]:
    k = H_K[h]
    print(f"\n=== Horizon h=+{h} days | window k={k} ===")
    X, y, ts = build_supervised(df, k=k, h=h)
    t_val, t_test = temporal_split(ts, test_frac=0.2, val_frac=0.2)
    (X_tr, y_tr), (X_va, y_va), (X_te, y_te) = split_by_time(X.values, y.values, ts, t_val, t_test)

    # Model: RandomForest
    base = RandomForestClassifier(class_weight='balanced', random_state=42, n_jobs=-1)
    search = RandomizedSearchCV(
        estimator=base,
        param_distributions=RFC_PARAM_DIST,
        n_iter=25,
        scoring='f1',
        cv=3,
        random_state=42,
        n_jobs=-1,
        verbose=1
    )
    search.fit(X_tr, y_tr)
    best = search.best_estimator_

    # Validation evaluation
    if hasattr(best, 'predict_proba'):
        y_va_prob = best.predict_proba(X_va)[:, 1]
        y_te_prob = best.predict_proba(X_te)[:, 1]
    else:
        # Fall back to decision function
        y_va_raw = best.decision_function(X_va)
        y_te_raw = best.decision_function(X_te)
        y_va_prob = (y_va_raw - y_va_raw.min()) / (y_va_raw.ptp() + 1e-9)
        y_te_prob = (y_te_raw - y_te_raw.min()) / (y_te_raw.ptp() + 1e-9)

    m_va = compute_metrics(y_va, y_va_prob)
    m_te = compute_metrics(y_te, y_te_prob)
    print('Validation:', m_va)
    print('Test:', m_te)

    # Calibration
    calibrator = CalibratedClassifierCV(best, method='sigmoid', cv=3)
    calibrator.fit(X_va, y_va)
    y_te_prob_cal = calibrator.predict_proba(X_te)[:, 1]
    m_te_cal = compute_metrics(y_te, y_te_prob_cal)
    print('Test (calibrated):', m_te_cal)

    # Save models
    joblib.dump(best, os.path.join(MODELS_DIR, f'best_model_t+{h}.pkl'))
    joblib.dump(calibrator, os.path.join(MODELS_DIR, f'best_model_t+{h}_calibrated.pkl'))

    # Curves
    fpr, tpr, _ = roc_curve(y_te, y_te_prob_cal)
    prec, rec, _ = precision_recall_curve(y_te, y_te_prob_cal)
    fig, ax = plt.subplots(1,2, figsize=(10,4))
    ax[0].plot(fpr, tpr, label=f'AUC={roc_auc_score(y_te, y_te_prob_cal):.3f}')
    ax[0].plot([0,1],[0,1],'--',c='gray'); ax[0].set_title(f'ROC t+{h}')
    ax[0].set_xlabel('FPR'); ax[0].set_ylabel('TPR'); ax[0].legend()
    ax[1].plot(rec, prec); ax[1].set_title(f'PR t+{h}')
    ax[1].set_xlabel('Recall'); ax[1].set_ylabel('Precision')
    plt.tight_layout(); plt.show()

    results.append({'horizon': h, 'val': m_va, 'test': m_te, 'test_cal': m_te_cal})

print('\nSummary:')
for r in results:
    print(r['horizon'], r['test_cal'])

In [ ]:
# 8) LSTM Sequence Model for t+H Classification
# Requires TensorFlow/Keras. If not installed, run:
# pip install tensorflow==2.15.0

try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers
    TF_AVAILABLE = True
    print('TensorFlow:', tf.__version__)
except Exception as e:
    TF_AVAILABLE = False
    print('TensorFlow not available. Install tensorflow to run LSTM section.')


def build_sequences(df_in: pd.DataFrame, k: int, h: int):
    """Return 3D arrays X_seq (N, k, F) and y (N,) aggregated across pixels."""
    feats = WINDOW_FEATURES  # could reduce for LSTM if needed
    X_list, y_list = [], []
    for _, g in df_in.groupby(GROUP_COLS):
        g = g.sort_values(DATE_COL)
        V = g[feats].values
        y_arr = g[TARGET_COL].values
        for t in range(k, len(g) - h):
            X_list.append(V[t-k:t, :])
            y_list.append(y_arr[t + h])
    X_seq = np.stack(X_list, axis=0)
    y = np.array(y_list).astype('int32')
    return X_seq, y

if TF_AVAILABLE:
    for h in [1,7,10]:
        k = H_K[h]
        print(f"\n[LSTM] Horizon h=+{h} | window k={k}")
        X_seq, y = build_sequences(df, k=k, h=h)

        # Temporal split by index
        n = len(y)
        i_val = int(n*0.6)
        i_test = int(n*0.8)
        X_tr, y_tr = X_seq[:i_val], y[:i_val]
        X_va, y_va = X_seq[i_val:i_test], y[i_val:i_test]
        X_te, y_te = X_seq[i_test:], y[i_test:]

        # Model
        model = keras.Sequential([
            layers.Input(shape=(k, X_seq.shape[-1])),
            layers.Masking(mask_value=0.0),
            layers.LSTM(64, return_sequences=False),
            layers.Dropout(0.2),
            layers.Dense(32, activation='relu'),
            layers.Dropout(0.2),
            layers.Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer=keras.optimizers.Adam(1e-3),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
        cb = [keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
        hist = model.fit(X_tr, y_tr, validation_data=(X_va, y_va), epochs=30, batch_size=256, callbacks=cb, verbose=1)

        y_te_prob = model.predict(X_te, verbose=0).ravel()
        m_te = compute_metrics(y_te, y_te_prob)
        print('[LSTM] Test:', m_te)

        # Save
        out_path = os.path.join(MODELS_DIR, f'lstm_t+{h}.keras')
        model.save(out_path)
        print('Saved:', out_path)

In [ ]:
# 9) Compare Lead-Time Performance & Plot
import pprint
pp = pprint.PrettyPrinter(indent=2)
print('\nTree-based results (per horizon, calibrated test):')
pp.pprint(results)

# 10) Notes & Next Steps (markdown below)

## Notes and Next Steps
- Start with tree ensembles + lag features; move to Temporal CNN/LSTM if 7–10 day performance lags.
- Add exogenous drivers: river discharge, wind stress, MLD, precipitation; align to timestamps.
- Use blocked/rolling-origin CV; never shuffle across time.
- Always compare to persistence and climatology baselines.
- Calibrate per-horizon probabilities (Platt scaling) and verify with Brier score + reliability diagrams.
- For raster inference, generate horizon-specific rasters using the same window builder and models saved as `best_model_t+H.pkl` or `lstm_t+H.keras`.